Hello and welcome to my capstone project notebook.
This notebook will be used to showcase the work that I've done to finish this class.

In [13]:
from geopy.geocoders import Nominatim
import folium
# Import the necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# requests for fetching html of website
import requests

In [14]:
geolocator = Nominatim(timeout=10, user_agent = "sje.Capstone-Project")
# Read the CSV using the City of Chicago data
stations = pd.read_csv("https://data.cityofchicago.org/api/views/8pix-ypme/rows.csv?accessType=DOWNLOAD")
stations.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


only blue line

In [15]:
blue_line = stations[ stations['BLUE'] == True ]
blue_line = pd.DataFrame(blue_line)
blue_line.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
33,30002,W,Austin (Forest Pk-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
34,30001,E,Austin (O'Hare-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
37,30013,S,Belmont (O'Hare Branch) (Forest Pk-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
38,30012,N,Belmont (O'Hare Branch) (O'Hare-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
61,30188,W,Cicero (Forest Pk-bound),Cicero,Cicero (Blue Line),40970,False,False,True,False,False,False,False,False,False,False,"(41.871574, -87.745154)"


remove duplicates

In [16]:
blue_line.drop_duplicates(subset ="Location", inplace = True)
blue_line.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
33,30002,W,Austin (Forest Pk-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
37,30013,S,Belmont (O'Hare Branch) (Forest Pk-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
61,30188,W,Cicero (Forest Pk-bound),Cicero,Cicero (Blue Line),40970,False,False,True,False,False,False,False,False,False,False,"(41.871574, -87.745154)"
67,30374,S,Clark/Lake (Forest Pk-bound),Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True,False,True,False,False,False,False,False,False,False,"(41.885737, -87.630886)"
69,30085,W,Clinton (Forest Pk-bound),Clinton,Clinton (Blue Line),40430,False,False,True,False,False,False,False,False,False,False,"(41.875539, -87.640984)"


In [17]:
blue_line['Latitude'], blue_line['Longitude'] = blue_line['Location'].str.split(' ', 1).str

<ipython-input-17-401db6cdd0c2>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  blue_line['Latitude'], blue_line['Longitude'] = blue_line['Location'].str.split(' ', 1).str


In [18]:
# Drop all the extra columns
blue_line.drop(['DIRECTION_ID', 'STATION_NAME', 'MAP_ID', 'STOP_NAME', 'ADA', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O'], axis = 1, inplace=True)
blue_line['STATION_DESCRIPTIVE_NAME'] = blue_line['STATION_DESCRIPTIVE_NAME'].str.replace('(Blue Line)', '', regex=False)
blue_line.rename(columns={'STATION_DESCRIPTIVE_NAME': 'Station'}, inplace=True)
blue_line.head()

,STOP_ID,Station,Location,Latitude,Longitude
33,30002,Austin,"(41.870851, -87.776812)","(41.870851,",-87.776812)
37,30013,Belmont,"(41.938132, -87.712359)","(41.938132,",-87.712359)
61,30188,Cicero,"(41.871574, -87.745154)","(41.871574,",-87.745154)
67,30374,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)","(41.885737,",-87.630886)
69,30085,Clinton,"(41.875539, -87.640984)","(41.875539,",-87.640984)


In [19]:
blue_line.set_index("STOP_ID", inplace=True)
blue_line.head()

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)","(41.870851,",-87.776812)
30013,Belmont,"(41.938132, -87.712359)","(41.938132,",-87.712359)
30188,Cicero,"(41.871574, -87.745154)","(41.871574,",-87.745154)
30374,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)","(41.885737,",-87.630886)
30085,Clinton,"(41.875539, -87.640984)","(41.875539,",-87.640984)


In [20]:
blue_line.at[30374,'Station'] = 'Clark/Lake'
#blue_line.loc[30374]['Station'] = 'Clark/Lake'

In [21]:
blue_line['Station'] = blue_line["Station"].str.replace('(Blue Line - O\'Hare Branch)', '(OH Branch)', regex=False)
blue_line['Station'] = blue_line["Station"].str.replace('(Blue Line - Forest Park Branch)', '(FB Branch)', regex=False)

In [22]:
blue_line

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)","(41.870851,",-87.776812)
30013,Belmont,"(41.938132, -87.712359)","(41.938132,",-87.712359)
30188,Cicero,"(41.871574, -87.745154)","(41.871574,",-87.745154)
30374,Clark/Lake,"(41.885737, -87.630886)","(41.885737,",-87.630886)
30085,Clinton,"(41.875539, -87.640984)","(41.875539,",-87.640984)
30045,Cumberland,"(41.984246, -87.838028)","(41.984246,",-87.838028)
30116,Damen,"(41.909744, -87.677437)","(41.909744,",-87.677437)
30063,Division,"(41.903355, -87.666496)","(41.903355,",-87.666496)
30076,Forest Park,"(41.874257, -87.817318)","(41.874257,",-87.817318)


In [23]:
blue_line['Longitude'] = blue_line["Longitude"].str.replace(')', '', regex=False)
blue_line['Latitude'] = blue_line["Latitude"].str.replace('(', '', regex=False)
blue_line['Latitude'] = blue_line["Latitude"].str.replace(',', '', regex=False)
blue_line[["Longitude"]] = blue_line[["Longitude"]].astype("float")
blue_line[["Latitude"]] = blue_line[["Latitude"]].astype("float")
blue_line.head()

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984


In [24]:
# Let's get the basic location data for Chicago, just to get started
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="sje_capstone-project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


Add zipcodes to dataframe for use later

In [25]:
# This function strips the parenthesis from the 'Location' column and returns a zipcode string
def getzip (location_info):
    
    zipcode = geolocator.geocode(location_info[1:-1], addressdetails=True).raw['address']['postcode']

    return zipcode[0:5]

In [26]:
blue_line['Zipcode'] = blue_line['Location'].apply(getzip)
blue_line.head()

,Station,Location,Latitude,Longitude,Zipcode
STOP_ID,,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812,60130
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359,60659
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154,60644
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886,60696
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984,60606


In [27]:
blue_line.head(10)

,Station,Location,Latitude,Longitude,Zipcode
STOP_ID,,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812,60130
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359,60659
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154,60644
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886,60696
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984,60606
30045,Cumberland,"(41.984246, -87.838028)",41.984246,-87.838028,60631
30116,Damen,"(41.909744, -87.677437)",41.909744,-87.677437,60647
30063,Division,"(41.903355, -87.666496)",41.903355,-87.666496,60642
30076,Forest Park,"(41.874257, -87.817318)",41.874257,-87.817318,60130


In [29]:
# create map of Chciago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['Station']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [30]:

# create map of Chciago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['Station']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)
    folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_chicago)
  
    
map_chicago


# CL Scraping Sections

In [31]:
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
from requests import get
from geopy import distance

In [32]:
# Create a list of zipcodes to scrape
blue_line.reset_index(drop=True, inplace=True)
blue_line.head()

,Station,Location,Latitude,Longitude,Zipcode
0,Austin,"(41.870851, -87.776812)",41.870851,-87.776812,60130
1,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359,60659
2,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154,60644
3,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886,60696
4,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984,60606


In [33]:
ziplist = blue_line['Zipcode']
ziplist = list(dict.fromkeys(ziplist))
print(ziplist)
print(str(len(ziplist)) + " unique zipcodes")

['60130', '60659', '60644', '60696', '60606', '60631', '60647', '60642', '60661', '60656', '60607', '60641', '60604', '60630', '60605', '60603', '60688', '60018', '60602', '60625', '60618']
21 unique zipcodes


In [34]:
# This function concats all of the search paramters into a long url
def get_cl_url (zipcode, distance, page, min_bed, max_bed):
    cl_url =("https://chicago.craigslist.org/search/apa?"
                   + "&search_distance="
                   + distance
                   + "&postal="
                   + zipcode
                   + "&min_bedrooms="
                   + min_bed
                   + "&max_bedrooms="
                   + max_bed
                   + "&s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&availabilityMode=0")
    return cl_url

In [35]:
get_cl_url('60642', '2', '0', '0', '1')

'https://chicago.craigslist.org/search/apa?&search_distance=2&postal=60642&min_bedrooms=0&max_bedrooms=1&s=0&availabilityMode=0'

In [36]:
# This function pulls the location data by scraping the page on a URL
def get_loc_from_listing (url):
  #get the latitude and longitude from a URL listing
  listingHtml = get(url)
  listSoup = BeautifulSoup(listingHtml.text, 'html.parser')
  location = listSoup.find_all(class_= 'viewposting')
  loc = []
  if location[0] is None:
      loc.append('null')
      loc.append('null')

  else:
      loc.append(float(location[0]['data-latitude']))
      loc.append(float(location[0]['data-longitude']))
  return loc

In [37]:
# Create empty arrays for storing data
post_title_texts = []
post_hoods = []
post_links = []
post_prices = []
longitudes = []
latitudes = []
post_zipcodes = []

for zipcode in ziplist:
    
    #get the first page of the chicago housing listings in this zipcode
    response = get(get_cl_url(zipcode,'2','0','0','1')) # 0 or 1 bedrooms 2 mi from zipcode

    html_soup = BeautifulSoup(response.text, 'html.parser')

    #get the macro-container for the housing posts
    posts = html_soup.find_all('li', class_= 'result-row')


    #find the total number of posts to find the limit of the pagination
    results_num = html_soup.find('div', class_= 'search-legend')
    results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

    #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
    pages = np.arange(0, results_total+1, 120)

    iterations = 0


    for page in pages:

        #get request
        response = get(get_cl_url(zipcode,'2', page,'0','1')) #Searches for one bedrooms or studios

        sleep(randint(1,5))

        #throw warning for status codes that are not 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        #define the html text
        page_html = BeautifulSoup(response.text, 'html.parser')

        #define the posts
        posts = html_soup.find_all('li', class_= 'result-row')

        #extract data item-wise
        for post in posts:

            if post.find('span', class_ = 'result-hood') is not None:

                #neighborhoods
                post_hood = post.find('span', class_= 'result-hood').text
                post_hoods.append(post_hood)

                #title text
                post_title = post.find('a', class_='result-title hdrlnk')
                post_title_text = post_title.text
                post_title_texts.append(post_title_text)

                #post link
                post_link = post_title['href']
                post_links.append(post_link)

                # Location Data
                bothloc = get_loc_from_listing(post_link)
                latitudes.append(bothloc[0])
                longitudes.append(bothloc[1])

                #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
                post_price = post.a.text.strip().replace("$", "")
                post_prices.append(post_price)
                
                # Records the zipcode it was searched by
                post_zipcodes.append(zipcode)


        iterations += 1
        print("Page " + str(iterations) + " of zipcode "+ zipcode + " scraped successfully!")


print("\n")

print("Scrape complete!")

Page 1 of zipcode 60130 scraped successfully!
Page 1 of zipcode 60659 scraped successfully!
Page 1 of zipcode 60644 scraped successfully!
Page 1 of zipcode 60696 scraped successfully!
Page 2 of zipcode 60696 scraped successfully!
Page 3 of zipcode 60696 scraped successfully!
Page 4 of zipcode 60696 scraped successfully!
Page 1 of zipcode 60606 scraped successfully!
Page 2 of zipcode 60606 scraped successfully!
Page 3 of zipcode 60606 scraped successfully!
Page 4 of zipcode 60606 scraped successfully!
Page 5 of zipcode 60606 scraped successfully!
Page 6 of zipcode 60606 scraped successfully!
Page 7 of zipcode 60606 scraped successfully!
Page 8 of zipcode 60606 scraped successfully!
Page 9 of zipcode 60606 scraped successfully!
Page 10 of zipcode 60606 scraped successfully!
Page 1 of zipcode 60631 scraped successfully!
Page 1 of zipcode 60647 scraped successfully!
Page 2 of zipcode 60647 scraped successfully!
Page 1 of zipcode 60642 scraped successfully!
Page 2 of zipcode 60642 scraped s

In [38]:
chi_apts = pd.DataFrame({'neighborhood': post_hoods,
                       'post title': post_title_texts,
                        'URL': post_links,
                       'price': post_prices,
                        'longitude': longitudes,
                        'latitude': latitudes})
chi_apts.head(10)

,neighborhood,post title,URL,price,longitude,latitude
0,(Forest Park),"1 bed 1 bath vintage apt, laundry, parking and...",https://chicago.craigslist.org/wcl/apa/d/fores...,,-87.810600,41.874400
1,(Forest Park),"BRIGHT 1 BED, SS Appliances with Balcony - MET...",https://chicago.craigslist.org/nch/apa/d/fores...,995,-87.810600,41.874400
2,(Forest Park),Studio ($690) and 1 Bedroom ($810/$850) Apartm...,https://chicago.craigslist.org/nwc/apa/d/studi...,,-87.810600,41.874400
3,(FOREST PARK),GIANT BALCONY & HEATED GARAGE 1 BDRM APT.,https://chicago.craigslist.org/wcl/apa/d/giant...,1100,-87.810600,41.874400
4,(Forest Park),SEMI-MODERN ONE BEDROOMS FROM $1005-1015 - 227,https://chicago.craigslist.org/wcl/apa/d/fores...,1005,-87.810600,41.874400
5,(Forest Park),"FOREST PARK / OAK PARK 1 Bedroom, Balcony, 601...",https://chicago.craigslist.org/nch/apa/d/fores...,985,-87.810600,41.874400
6,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400
7,"(Forest Park, IL)","7219 Madison, Forest Park",https://chicago.craigslist.org/wcl/apa/d/fores...,,-87.810600,41.874400
8,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1125,-87.814258,41.875831
9,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1125,-87.814258,41.875831


In [46]:
chi_apts.shape

(924, 6)

In [40]:
# Drops any apartments missing price data
chi_apts.drop( chi_apts[ chi_apts['price'] == '' ].index , inplace=True)

Data Cleaning of all apartments dataframe


In [42]:

# Convert prices to int
chi_apts['price'] = chi_apts['price'].apply(lambda x: int(x))

In [48]:
# Drop any apartments less than 300 (often spam)
chi_apts.drop( chi_apts[ chi_apts['price'] <500 ].index , inplace=True)

In [45]:
# Remove duplicates with same URL - these are relistings
chi_apts = chi_apts.drop_duplicates(subset='URL')

In [49]:
chi_apts.sort_values(['price'], inplace=True)



chi_apts.reset_index(drop=True, inplace=True)

chi_apts.head(30)

,neighborhood,post title,URL,price,longitude,latitude
0,(McKinley Park),"1br apt , McKinley Park",https://chicago.craigslist.org/chc/apa/d/chica...,675,-87.669400,41.851500
1,(Austin),Completely updated garden unit all utilities i...,https://chicago.craigslist.org/nch/apa/d/chica...,700,-87.756116,41.903149
2,(Berwyn),Studio apartment with full-size kitchen!,https://chicago.craigslist.org/wcl/apa/d/chica...,700,-87.793872,41.857767
3,(Irving Park),Studio Apt Available March 1st,https://chicago.craigslist.org/chc/apa/d/chica...,720,-87.729821,41.955918
4,(3353 W WARREN BLVD),Section 8 Welcome! Long Style Studio! ALL UTIL...,https://chicago.craigslist.org/chc/apa/d/chica...,725,-87.722300,41.880400
5,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400
6,(McKinley Park),Newly renovated studio with separate kitchen -...,https://chicago.craigslist.org/chc/apa/d/chica...,750,-87.669400,41.851500
7,(7301 N. Ridge),Rogers Park Studio On Ridge Blvd.,https://chicago.craigslist.org/chc/apa/d/chica...,750,-87.682557,42.010231
8,(Chicago`),Studio Chicago/Oak Park Border $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774057,41.867231
9,(Harvard and Austin),Oak Park Studio $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774157,41.869131


In [52]:
chi_apts.shape

(842, 6)

In [51]:
# Remove duplicates with same post titles and price - these are most likely relistings
chi_apts = chi_apts.drop_duplicates(subset=['post title', 'price'])

In [53]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude
0,(McKinley Park),"1br apt , McKinley Park",https://chicago.craigslist.org/chc/apa/d/chica...,675,-87.669400,41.851500
1,(Austin),Completely updated garden unit all utilities i...,https://chicago.craigslist.org/nch/apa/d/chica...,700,-87.756116,41.903149
2,(Berwyn),Studio apartment with full-size kitchen!,https://chicago.craigslist.org/wcl/apa/d/chica...,700,-87.793872,41.857767
3,(Irving Park),Studio Apt Available March 1st,https://chicago.craigslist.org/chc/apa/d/chica...,720,-87.729821,41.955918
4,(3353 W WARREN BLVD),Section 8 Welcome! Long Style Studio! ALL UTIL...,https://chicago.craigslist.org/chc/apa/d/chica...,725,-87.722300,41.880400


In [54]:
chi_apts.shape

(842, 6)

In [76]:
for index, row in blue_line.iterrows(): 
    print(row['Station'])

Austin 
Belmont 
Cicero 
Clark/Lake (Blue, Brown, Green, Orange, Purple & Pink Lines)
Clinton 
Cumberland 
Damen 
Division 
Forest Park 
Grand 
Harlem (Blue Line - O'Hare Branch)
Harlem (Blue Line - Forest Park Branch)
Illinois Medical District 
Irving Park 
Jackson 
Jefferson Park 
Kedzie-Homan 
LaSalle 
Logan Square 
Montrose 
Monroe 
Oak Park 
Pulaski 
Racine 
O'Hare 
Rosemont 
UIC-Halsted 
Washington 
California 
Chicago 
Western (Blue Line - O'Hare Branch)
Western (Blue Line - Forest Park Branch)
Addison 


In [55]:
chi_apts['Nearby Stations'] = 'None'

In [123]:
blue_line.head()

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984


In [56]:
for index, row in blue_line.iterrows(): 
    station_lat = row['Latitude']
    station_long = row['Longitude']
    station_name = row['Station']
    for index, apt_row in chi_apts.iterrows():
        if distance.distance((station_lat, station_long), (apt_row.latitude, apt_row.longitude)).m < 600:
            #print("found one")
            #print(apt_row['Nearby Stations'])
            if chi_apts.at[index,'Nearby Stations'] == 'None':
                chi_apts.at[index,'Nearby Stations'] = station_name
            else:
                chi_apts.at[index,'Nearby Stations'] += (", " + station_name)
   #print(str(station_lat) + " " + str(station_long))

In [57]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations
0,(McKinley Park),"1br apt , McKinley Park",https://chicago.craigslist.org/chc/apa/d/chica...,675,-87.669400,41.851500,None
1,(Austin),Completely updated garden unit all utilities i...,https://chicago.craigslist.org/nch/apa/d/chica...,700,-87.756116,41.903149,None
2,(Berwyn),Studio apartment with full-size kitchen!,https://chicago.craigslist.org/wcl/apa/d/chica...,700,-87.793872,41.857767,None
3,(Irving Park),Studio Apt Available March 1st,https://chicago.craigslist.org/chc/apa/d/chica...,720,-87.729821,41.955918,Irving Park
4,(3353 W WARREN BLVD),Section 8 Welcome! Long Style Studio! ALL UTIL...,https://chicago.craigslist.org/chc/apa/d/chica...,725,-87.722300,41.880400,None


In [58]:
# Drops any apartments not close to a train
chi_apts.drop( chi_apts[ chi_apts['Nearby Stations'] == 'None' ].index , inplace=True)

In [59]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations
3,(Irving Park),Studio Apt Available March 1st,https://chicago.craigslist.org/chc/apa/d/chica...,720,-87.729821,41.955918,Irving Park
5,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)"
8,(Chicago`),Studio Chicago/Oak Park Border $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774057,41.867231,Austin
9,(Harvard and Austin),Oak Park Studio $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774157,41.869131,Austin
12,(Chicago),Large Fresh!! 1BDR Lower Unit w/Carpet! New Li...,https://chicago.craigslist.org/chc/apa/d/chica...,765,-87.632100,41.882900,"Clark/Lake, Jackson , Monroe , Washington"


In [66]:
chi_apts.shape

(350, 8)

In [68]:
chi_apts.tail()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Rent
913,(West Loop / Chicago),"Furnished, Comfy West Loop 1BR w/ Gym, Pool ne...",https://chicago.craigslist.org/chc/apa/d/chica...,2890,-87.646019,41.879232,"Clinton , UIC-Halsted",med
914,(The Loop),"Furnished, Dapper Loop 1BR w/ Gym, Pool, W/D n...",https://chicago.craigslist.org/chc/apa/d/chica...,2890,-87.634098,41.883256,"Clark/Lake, Monroe , Washington",med
916,(The Loop),"Furnished, Chic Loop 1BR w/ Gym, Pool, W/D, by...",https://chicago.craigslist.org/chc/apa/d/chica...,2990,-87.628789,41.884529,"Clark/Lake, Monroe , Washington",med
919,(Chicago),Fully Furnished - Month to Month - 1 Bedroom a...,https://chicago.craigslist.org/chc/apa/d/chica...,3500,-87.631470,41.887042,"Clark/Lake, Washington",med
920,(Chicago),"Beautiful Townhouse w/ Hardwood Floor, AMAZING...",https://chicago.craigslist.org/chc/apa/d/chica...,6000,-87.627700,41.871300,LaSalle,high


In [61]:
austin = chi_apts[chi_apts['Nearby Stations'].str.contains("Austin")]
austin.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations
8,(Chicago`),Studio Chicago/Oak Park Border $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774057,41.867231,Austin
9,(Harvard and Austin),Oak Park Studio $750,https://chicago.craigslist.org/wcl/apa/d/oak-p...,750,-87.774157,41.869131,Austin
63,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin
71,(OAK PARK),Arts District Studio w/ ALL UTILITIES + INTERNET,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.775824,41.872731,Austin
80,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin


In [62]:
austin.mean().price

950.0

In [63]:
# Rates the price of the apartment
chi_apts['Rent'] = pd.cut(chi_apts['price'], bins=3, labels=['low', 'med', 'high'])

In [64]:
chi_apts['Rent'].value_counts()

low     324
med      25
high      1
Name: Rent, dtype: int64

In [169]:
austin = chi_apts[chi_apts['Nearby Stations'].str.contains("Austin")]
austin.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations,Rent
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin,low
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin,low
62,(Oak Park),VINTAGE ONE BEDROOM - 900,https://chicago.craigslist.org/wcl/apa/d/oak-p...,1450,-87.777957,41.872658,Austin,Austin Austin Austin Austin,high


In [171]:
# Builds empty dataframe for storing all data on each train stop (our goal!)
columns = ['Train Stop', 'Avg Rent', 'Num Apts', 'Cheap', 'Med', 'Luxury', 'Bars']
stop_data = pd.DataFrame(columns=columns)

# Stores the names of each stop in a row
stop_data['Train Stop'] = blue_line['Station']

In [190]:
stop_data.head()

,Train Stop,Avg Rent,Num Apts,Cheap,Med,Luxury,Bars
STOP_ID,,,,,,,
30002,Austin,1091,NaN,NaN,NaN,NaN,NaN
30013,Belmont,1091,NaN,NaN,NaN,NaN,NaN
30188,Cicero,1091,NaN,NaN,NaN,NaN,NaN
30374,Clark/Lake,1091,NaN,NaN,NaN,NaN,NaN
30085,Clinton,1091,NaN,NaN,NaN,NaN,NaN


In [189]:
stop_data['Avg Rent'] = int(chi_apts[chi_apts['Nearby Stations'].str.contains('Austin')].mean().price)

In [186]:
stop_data['Train Stop']

STOP_ID
30002                       Austin 
30013                      Belmont 
30188                       Cicero 
30374                    Clark/Lake
30085                      Clinton 
30045                   Cumberland 
30116                        Damen 
30063                     Division 
30076                  Forest Park 
30096                        Grand 
30146              Harlem OH Branch
30189            Harlem (FB Branch)
30158    Illinois Medical District 
30108                  Irving Park 
30015                      Jackson 
30248               Jefferson Park 
30049                 Kedzie-Homan 
30262                      LaSalle 
30198                 Logan Square 
30260                     Montrose 
30154                       Monroe 
30035                     Oak Park 
30180                      Pulaski 
30093                       Racine 
30172                       O'Hare 
30160                     Rosemont 
30069                  UIC-Halsted 
30073               

In [27]:
chi_apts.iloc[0].URL

'https://chicago.craigslist.org/chc/apa/d/chicago-luxurious-1-bed-1-bath-condo-in/7065399594.html'

In [166]:
print(distance.distance((damen_lat, damen_long), (chi_apts.iloc[16].latitude, chi_apts.iloc[16].longitude)).m)

523.414145424127


In [251]:
chi_apts['Stop'] = 'Damen'
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop
0,(Chicago - Bucktown),Rehabbed Studio Loft! Granite/Stainless. Hardw...,https://chicago.craigslist.org/chc/apa/d/chica...,1650,-87.7043,41.9209,Damen
1,(Chicago - Lincoln Park),Great Value 1BR with WD in unit in Lincoln Park!,https://chicago.craigslist.org/chc/apa/d/chica...,1525,-87.7043,41.9209,Damen
2,(Dickens & Central Park),"Choice 1 Bedroom, 1 Bath in Logan Square, Avai...",https://chicago.craigslist.org/chc/apa/d/chica...,1350,-87.7043,41.9209,Damen
3,(Logan Square),Logan Square 1 bedroom heat included,https://chicago.craigslist.org/chc/apa/d/chica...,1095,-87.7043,41.9209,Damen
4,(Chicago),"Cute, Logan Square, free laundry, central air,...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.7043,41.9209,Damen


In [252]:
def distance_to_train (stop_lat, stop_long, train_lat = damen_lat, train_long = damen_long):
    return distance.distance((train_lat, train_long), (stop_lat, stop_long)).m

In [174]:
chi_apts.iloc[1].latitude

41.9019

In [253]:
chi_apts['Distance to train'] = chi_apts.apply(lambda x: distance_to_train(x.latitude, x.longitude), axis=1)

In [254]:
chi_apts.head(30)

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Chicago - Bucktown),Rehabbed Studio Loft! Granite/Stainless. Hardw...,https://chicago.craigslist.org/chc/apa/d/chica...,1650,-87.704300,41.920900,Damen,2549.886832
1,(Chicago - Lincoln Park),Great Value 1BR with WD in unit in Lincoln Park!,https://chicago.craigslist.org/chc/apa/d/chica...,1525,-87.704300,41.920900,Damen,2549.886832
2,(Dickens & Central Park),"Choice 1 Bedroom, 1 Bath in Logan Square, Avai...",https://chicago.craigslist.org/chc/apa/d/chica...,1350,-87.704300,41.920900,Damen,2549.886832
3,(Logan Square),Logan Square 1 bedroom heat included,https://chicago.craigslist.org/chc/apa/d/chica...,1095,-87.704300,41.920900,Damen,2549.886832
4,(Chicago),"Cute, Logan Square, free laundry, central air,...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.704300,41.920900,Damen,2549.886832
5,(Chicago),Extremely spacious one bedroom with one bathro...,https://chicago.craigslist.org/chc/apa/d/chica...,1499,-87.704300,41.920900,Damen,2549.886832
6,(Wicker Park),1.5 MONTHS FREE on this 1 bed 1 bath Blue Line...,https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.704300,41.920900,Damen,2549.886832
7,(Humboldt Park),Affordable Housing for residents with limited ...,https://chicago.craigslist.org/chc/apa/d/chica...,897,-87.704300,41.920900,Damen,2549.886832
8,(Lakeview),Luxury One Bedroom,https://chicago.craigslist.org/chc/apa/d/chica...,2350,-87.704300,41.920900,Damen,2549.886832
9,(Chicago),Bucktown- Armitage/ Damen,https://chicago.craigslist.org/chc/apa/d/chica...,1150,-87.704300,41.920900,Damen,2549.886832


In [255]:
chi_apts.sort_values(['Distance to train'], inplace=True)
chi_apts.reset_index(drop=True, inplace=True)
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
1,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
2,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
3,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
4,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029


In [256]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
4,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029
8,(Wicker Park),"Junior 1-bedroom in heart of Wicker, steps to ...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.677554,41.911027,Damen,142.835023
12,"(6 Corners, Wicker Park, Chicago)","Wicker Park 1 Bed, Utilities Included - Fully ...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.674854,41.908632,Damen,247.351596
20,(Wicker Park),Large 1.5 bedroom in Wicker Park,https://chicago.craigslist.org/chc/apa/d/chica...,1930,-87.680009,41.911232,Damen,269.909557


In [257]:
chi_apts.shape

(91, 8)

In [258]:
chi_apts.sort_values(['Distance to train'], inplace=True)
chi_apts.reset_index(drop=True, inplace=True)
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
1,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029
2,(Wicker Park),"Junior 1-bedroom in heart of Wicker, steps to ...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.677554,41.911027,Damen,142.835023
3,"(6 Corners, Wicker Park, Chicago)","Wicker Park 1 Bed, Utilities Included - Fully ...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.674854,41.908632,Damen,247.351596
4,(Wicker Park),Large 1.5 bedroom in Wicker Park,https://chicago.craigslist.org/chc/apa/d/chica...,1930,-87.680009,41.911232,Damen,269.909557


In [259]:
chi_apts.drop( chi_apts[ chi_apts['Distance to train'] >600 ].index , inplace=True)
chi_apts.tail()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
6,(Chicago),*Gut Rehabbed* 1BED+DEN/1BTH In the Heart of B...,https://chicago.craigslist.org/chc/apa/d/chica...,2100,-87.680566,41.911932,Damen,355.603617
7,(WOLCOTT AVENUE),***NEW REDUCED PRICE*** HEART OF BUCKTOWN-ONE ...,https://chicago.craigslist.org/chc/apa/d/chica...,1550,-87.675254,41.912993,Damen,403.770377
8,(Chicago),"Bucktown - Spacious 1 B.R., Extra Large Kitche...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.673183,41.912332,Damen,455.186005
9,(2007 W. Potomac- #2),Wicker Park Apartment with EVERYTHING! (2007-#2),https://chicago.craigslist.org/chc/apa/d/chica...,1975,-87.677354,41.905032,Damen,523.414145
10,(CHICAGO),BUCKTOWN 1 BED TIMBER LOFT- PARKING INCLUDED- ...,https://chicago.craigslist.org/chc/apa/d/buckt...,2200,-87.680155,41.914732,Damen,598.157905


In [260]:
chi_apts.shape

(11, 8)

In [201]:
chi_apts.iloc[11].URL

'https://chicago.craigslist.org/chc/apa/d/chicago-1-month-free-no-deposit-best-of/7077312182.html'

In [261]:
# create map of Chciago using latitude and longitude values
map_damen = folium.Map(location=[damen_lat, damen_long], zoom_start=15)

# add station markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['STATION_NAME']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_damen)
    folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_damen)

# add apartment markers to map
for lat, lng, price in zip(chi_apts['latitude'], chi_apts['longitude'], chi_apts['price']):
    label = price
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_damen)
folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_damen)  
    
map_damen

The geograpical coordinate of Chicago are 41.8925325, -87.6576948.
